In [2]:
import requests
import sys
import json
import base64
import time
import logging
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction 
from solana.rpc.api import Client
from solana.rpc.types import TxOpts
import dontshare as d 
from pprint import pprint
from functools import lru_cache
from datetime import datetime, timedelta
import pytz
import json
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML
import time

In [3]:
API_Key = d.birdeye
wallet = d.sol_wallet
chain = "solana"
multichain = "solana,ethereum,bsc"

### DEFI APIs

OHLCV Data

In [4]:
chain = "solana"
address = "EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm" #WIF
type = "5m"

# Available timeframes:
# "1m"  - 1 minute
# "3m"  - 3 minutes
# "5m"  - 5 minutes
# "15m" - 15 minutes
# "30m" - 30 minutes
# "1H"  - 1 hour
# "2H"  - 2 hours
# "4H"  - 4 hours
# "6H"  - 6 hours
# "8H"  - 8 hours
# "12H" - 12 hours
# "1D"  - 1 day

# Human-readable date and time inputs
start_datetime = "2024-05-19 10:00:00"
end_datetime = "2024-05-19 11:00:00"

# Convert to datetime objects (assuming UTC timezone)
start_dt = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
end_dt = datetime.strptime(end_datetime, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)

# Convert to Unix timestamps
time_from = int(start_dt.timestamp())
time_to = int(end_dt.timestamp())

url = f"https://public-api.birdeye.so/defi/ohlcv?address={address}&type={type}&time_from={time_from}&time_to={time_to}"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    try:
        # Parse the JSON response
        data = json.loads(response.text)
        
        # Check if 'data' and 'items' keys exist
        if 'data' in data and 'items' in data['data']:
            items = data['data']['items']
            
            if items:  # Check if items list is not empty
                # Convert the list of dictionaries to a pandas DataFrame
                df = pd.DataFrame(items)

                # Convert 'unixTime' to datetime
                df['datetime'] = pd.to_datetime(df['unixTime'], unit='s')

                # Reorder columns to have 'datetime' first
                columns_order = ['datetime'] + [col for col in df.columns if col != 'datetime']
                df = df[columns_order]

                # Style the DataFrame
                styled_df = df.style.set_properties(**{
                    'white-space': 'nowrap',
                    'font-family': 'monospace',
                    'font-size': '10pt',
                    'text-align': 'right'
                }).set_table_styles([
                    {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
                    {'selector': 'td', 'props': [('padding', '5px')]}
                ]).format({
                    'datetime': lambda x: x.strftime('%Y-%m-%d %H:%M:%S'),
                    'o': '{:.2f}',
                    'h': '{:.2f}',
                    'l': '{:.2f}',
                    'c': '{:.2f}',
                    'v': '{:.2f}'
                })

                # Display the styled DataFrame
                display(HTML("<h3>OHLCV Data</h3>"))
                display(HTML(styled_df.to_html()))
            else:
                display(HTML("<h3>No data items found in the response.</h3>"))
                display(HTML("<pre>" + json.dumps(data, indent=2) + "</pre>"))
        else:
            display(HTML("<h3>Unexpected response structure. 'data' or 'items' key not found.</h3>"))
            display(HTML("<pre>" + json.dumps(data, indent=2) + "</pre>"))
    except json.JSONDecodeError:
        display(HTML("<h3>Failed to parse JSON response.</h3>"))
        display(HTML("<pre>" + response.text + "</pre>"))
else:
    display(HTML(f"<h3>Request failed with status code: {response.status_code}</h3>"))
    display(HTML("<pre>" + response.text + "</pre>"))

,datetime,address,c,h,l,o,type,unixTime,v
0,2024-05-19 10:00:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.57,2.58,2.56,2.57,5m,1716112800,12346.23
1,2024-05-19 10:05:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.57,2.58,2.56,2.57,5m,1716113100,9304.80
2,2024-05-19 10:10:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.56,2.57,2.56,2.57,5m,1716113400,20782.21
3,2024-05-19 10:15:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.56,2.58,2.56,2.56,5m,1716113700,15130.49
4,2024-05-19 10:20:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.57,2.58,2.56,2.56,5m,1716114000,108256.02
5,2024-05-19 10:25:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.57,2.58,2.57,2.57,5m,1716114300,46453.92
6,2024-05-19 10:30:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.58,2.58,2.56,2.57,5m,1716114600,21132.60
7,2024-05-19 10:35:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.59,2.59,2.58,2.58,5m,1716114900,53580.28
8,2024-05-19 10:40:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.59,2.60,2.58,2.59,5m,1716115200,84356.83
9,2024-05-19 10:45:00,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2.57,2.60,2.57,2.59,5m,1716115500,43059.84


### Token APIs

Market Data

In [9]:
def get_token_market_data(address, API_Key):
    """
    Fetch market data for a specific token and return it as a pandas DataFrame.

    Args:
    address (str): The token address to fetch market data for.
    API_Key (str): Your Birdeye API key.

    Returns:
    pd.DataFrame: A DataFrame containing the token's market data.
    """
    url = f"https://public-api.birdeye.so/defi/v3/token/market-data?address={address}"

    headers = {
        "accept": "application/json",
        "X-API-KEY": API_Key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if 'data' in data and data['success']:
            # Convert the data to a DataFrame
            df = pd.DataFrame([data['data']])
            
            # Rename columns for clarity
            df = df.rename(columns={
                'address': 'Address',
                'price': 'Price',
                'liquidity': 'Liquidity',
                'supply': 'Total Supply',
                'marketcap': 'Market Cap',
                'circulating_supply': 'Circulating Supply',
                'circulating_marketcap': 'Circulating Market Cap'
            })
            
            return df
        else:
            print("Unexpected response structure or request unsuccessful")
            return pd.DataFrame()
    else:
        print(f"Request failed with status code: {response.status_code}")
        return pd.DataFrame()

# Example usage:
address = "So11111111111111111111111111111111111111112"
market_data_df = get_token_market_data(address, API_Key)
market_data_df

,Token Address,Price,Liquidity,Total Supply,Market Cap,Circulating Supply,Circulating Market Cap
0,So11111111111111111111111111111111111111112,159.772739,1.570613e+10,5.868499e+08,9.376262e+10,5.868499e+08,9.376262e+10


Token List

In [21]:
def get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens):
    """
    Fetches a list of tokens from the Birdeye API based on specified filters and extracts selected fields.

    Args:
        sort_by (str): The field to sort by (e.g., "mc", "v24hUSD").
        sort_type (str): The sort order ("asc" or "desc").
        min_liquidity (float): Minimum liquidity in USD.
        min_volume_24h (float): Minimum 24-hour trading volume in USD.
        min_market_cap (float): Minimum market capitalization in USD.
        max_market_cap (float): Maximum market capitalization in USD.
        total_tokens (int): Number of tokens to retrieve.

    Returns:
        list: A list of dictionaries containing the selected token data.
    """
    limit = 50
    all_tokens = []

    for offset in range(0, total_tokens * 3, limit):  # Fetch more tokens to account for filtering
        url = (
            f"https://public-api.birdeye.so/defi/tokenlist?"
            f"sort_by={sort_by}&sort_type={sort_type}&offset={offset}"
            f"&limit={limit}&min_liquidity={min_liquidity}"
        )

        headers = {
            "accept": "application/json",
            "x-chain": chain,
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'tokens' in data['data']:
                new_tokens = [
                    {
                        'address': token.get('address'),
                        'liquidity': token.get('liquidity'),
                        'mc': token.get('mc'),
                        'name': token.get('name'),
                        'symbol': token.get('symbol'),
                        'v24hChangePercent': token.get('v24hChangePercent'),
                        'v24hUSD': token.get('v24hUSD')
                    }
                    for token in data['data']['tokens']
                    if token.get('v24hUSD', 0) >= min_volume_24h
                    and min_market_cap <= token.get('mc', 0) <= max_market_cap
                ]
                all_tokens.extend(new_tokens)
            else:
                print(f"Unexpected response structure at offset {offset}")
                break
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            break

        if len(all_tokens) >= total_tokens:
            break

        # Respect rate limits
        time.sleep(1)

    return all_tokens[:total_tokens]

# Example usage
sort_by = "mc"  # Options: rank, v24hUSD, v24hChangePercent 
sort_type = "desc"  # Options: asc, desc
min_liquidity = 100000  # Minimum liquidity in USD (100k)
min_volume_24h = 1000000  # Minimum 24-hour trading volume in USD
min_market_cap = 1000000  # Minimum market cap in USD (1 million)
max_market_cap = 1000000000  # Maximum market cap in USD (1 billion)
total_tokens = 250  # Number of tokens to retrieve

tokens = get_token_list(
    sort_by=sort_by,
    sort_type=sort_type,
    min_liquidity=min_liquidity,
    min_volume_24h=min_volume_24h,
    min_market_cap=min_market_cap,
    max_market_cap=max_market_cap,
    total_tokens=total_tokens
)

if tokens:
    df = pd.DataFrame(tokens)

    # Select and rename columns
    columns_to_display = {
        'address': 'Address',
        'liquidity': 'Liquidity',
        'mc': 'Market Cap',
        'name': 'Name',
        'symbol': 'Symbol',
        'v24hChangePercent': '24h Change (%)',
        'v24hUSD': 'Volume 24h (USD)'
    }
    
    # Only include columns that exist in the DataFrame
    available_columns = [col for col in columns_to_display.keys() if col in df.columns]
    df = df[available_columns].rename(columns={col: columns_to_display[col] for col in available_columns})

    # Format numeric columns
    df['Liquidity'] = df['Liquidity'].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else x)
    df['Market Cap'] = df['Market Cap'].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else x)
    df['24h Change (%)'] = df['24h Change (%)'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else x)
    df['Volume 24h (USD)'] = df['Volume 24h (USD)'].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else x)

    # Display the DataFrame
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', None)        # No line wrapping
    pd.set_option('display.max_colwidth', None) # Show full content

    display(df)
else:
    print("No token data available")

,Address,Liquidity,Market Cap,Name,Symbol,24h Change (%),Volume 24h (USD)
0,7LFeJiV7cfQhwpxUEECpGKmBisfPWkL8FZXFUFBbka5b,"$239,674,650.85","$999,710,087.96",USD Ccoin,USDC,-27.17%,"$150,641,674.75"
1,JAmWYLiv3pGnNz63UTSKSEmJG4pdvvTLCZUcmpSLJA4Q,"$101,180,309.39","$999,677,028.87",USD C,USDC,-15.65%,"$17,377,502.51"
2,HdoBnUg5bK3QB1BC4Mqz14mY5VmUufjvUMQjQmvpfSmh,"$97,105,001.73","$999,671,938.63",USD C,USDC,-18.90%,"$44,719,860.74"
3,Ht8Sgrn3mABFXjg26ivNozMZuv7xaRBJQJ1TeNUboMxZ,"$219,207,233.47","$999,652,985.84",USD C,USDC,45.10%,"$97,602,060.76"
4,DWEmswm2X2L3wbTmEfoyt5LqTtvFRha6HnNUfWfThuS6,"$120,114,082.42","$999,282,717.68",USD C,USDC,-30.29%,"$67,033,307.78"
5,KMNo3nJsBXfcpJTVhZcXLW7RmTwTt4GVFE7suUBo9sS,"$7,235,663.98","$940,559,824.23",Kamino,KMNO,-80.80%,"$1,009,482.59"
6,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,"$62,732,062.38","$808,428,066.51",Marinade staked SOL (mSOL),mSOL,-20.88%,"$15,023,201.83"
7,MEW1gQWJ3nEXg2qgERiKu7FAFj79PHvQVREQUzScPP5,"$52,335,308.23","$808,419,768.11",cat in a dogs world,MEW,-18.59%,"$19,196,207.90"
8,5LUx8Cmu61aCa9h8AebkmpqBBbgz4sErkbkGJESJXcFP,"$1,123,377,882.52","$751,805,108.23",AutoLaye,AutoLaye,-27.83%,"$318,068,185.76"
9,27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4,"$15,097,919.80","$748,342,424.10",Jupiter Perps LP,JLP,-45.01%,"$29,960,366.20"


In [ ]:
sort_by = "mc" #rank, v24hUSD, v24hChangePercent 
sort_type   = "desc" #asc, desc
offset = 0
limit = 50
min_liquidity = 10000

url = f"https://public-api.birdeye.so/defi/tokenlist?sort_by={sort_by}&sort_type={sort_type}&offset={offset}&limit={limit}&min_liquidity={min_liquidity}"

headers = {
    "accept": "application/json",
    "x-chain": chain,
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

print(response.text)

Token Security

In [4]:
url = f"https://public-api.birdeye.so/defi/token_security?address={address}"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = json.loads(response.text)
    if 'data' in data:
        # Extract the specific values
        token_data = data['data']
        selected_data = {
            'top10HolderPercent': token_data.get('top10HolderPercent', 'N/A'),
            'top10UserPercent': token_data.get('top10UserPercent', 'N/A'),
            'preMarketHolder': token_data.get('preMarketHolder', 'N/A'),
            'mutableMetadata': token_data.get('mutableMetadata', 'N/A'),
            'creatorPercentage': token_data.get('creatorPercentage', 'N/A'),
            'ownerPercentage': token_data.get('ownerPercentage', 'N/A')
        }

        # Create a DataFrame from the selected data
        df_selected = pd.DataFrame.from_dict(selected_data, orient='index', columns=['Value'])
        df_selected.index.name = 'Attribute'
        df_selected.reset_index(inplace=True)

        # Style the DataFrame
        styled_df = df_selected.style.set_properties(**{
            'white-space': 'pre-wrap',
            'font-family': 'monospace',
            'font-size': '10pt',
            'text-align': 'left'
        }).set_table_styles([
            {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'left')]},
            {'selector': 'td', 'props': [('text-align', 'left')]}
        ])

        # Display the styled DataFrame
        display(HTML("<h3>Selected Token Security Data</h3>"))
        display(HTML(styled_df.to_html()))

        # Display the full data as before
        df = pd.json_normalize(data['data'])
        df = df.map(lambda x: json.dumps(x, indent=2) if isinstance(x, (dict, list)) else x)
        df_pivoted = df.T.reset_index()
        df_pivoted.columns = ['Attribute', 'Value']
        styled_full_df = df_pivoted.style.set_properties(**{
            'white-space': 'pre-wrap',
            'font-family': 'monospace',
            'font-size': '10pt',
            'text-align': 'left'
        }).set_table_styles([
            {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'left')]},
            {'selector': 'td', 'props': [('text-align', 'left')]}
        ])
        #display(HTML("<h3>Full Token Security Data</h3>"))
        #display(HTML(styled_full_df.to_html()))
    else:
        display(HTML("<h3>Unexpected response structure. 'data' key not found.</h3>"))
        display(HTML("<pre>" + json.dumps(data, indent=2) + "</pre>"))
else:
    display(HTML(f"<h3>Request failed with status code: {response.status_code}</h3>"))
    display(HTML("<pre>" + response.text + "</pre>"))

,Attribute,Value
0,top10HolderPercent,0.392360
1,top10UserPercent,0.392360
2,preMarketHolder,[]
3,mutableMetadata,False
4,creatorPercentage,0.000000
5,ownerPercentage,None


Token Overview

In [5]:
url = f"https://public-api.birdeye.so/defi/token_overview?address={address}"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = json.loads(response.text)
    if 'data' in data:
        token_data = data['data']
        selected_data = {
            'address': token_data.get('address', 'N/A'),
            'decimals': token_data.get('decimals', 'N/A'),
            'symbol': token_data.get('symbol', 'N/A'),
            'name': token_data.get('name', 'N/A'),
            'price': token_data.get('price', 'N/A'),
            'liquidity': token_data.get('liquidity', 'N/A'),
            'uniqueWallet30m': token_data.get('uniqueWallet30m', 'N/A'),
            'uniqueWallet30mChangePercent': token_data.get('uniqueWallet30mChangePercent', 'N/A'),
            'uniqueWallet1h': token_data.get('uniqueWallet1h', 'N/A'),
            'uniqueWallet1hChangePercent': token_data.get('uniqueWallet1hChangePercent', 'N/A'),
            'uniqueWallet2h': token_data.get('uniqueWallet2h', 'N/A'),
            'uniqueWallet2hChangePercent': token_data.get('uniqueWallet2hChangePercent', 'N/A'),
            'uniqueWallet4h': token_data.get('uniqueWallet4h', 'N/A'),
            'uniqueWallet4hChangePercent': token_data.get('uniqueWallet4hChangePercent', 'N/A'),
            'uniqueWallet6h': token_data.get('uniqueWallet6h', 'N/A'),
            'uniqueWallet6hChangePercent': token_data.get('uniqueWallet6hChangePercent', 'N/A'),
            'uniqueWallet8h': token_data.get('uniqueWallet8h', 'N/A'),
            'uniqueWallet8hChangePercent': token_data.get('uniqueWallet8hChangePercent', 'N/A'),
            'uniqueWallet12h': token_data.get('uniqueWallet12h', 'N/A'),
            'uniqueWallet12hChangePercent': token_data.get('uniqueWallet12hChangePercent', 'N/A'),
            'uniqueWallet24h': token_data.get('uniqueWallet24h', 'N/A'),
            'uniqueWallet24hChangePercent': token_data.get('uniqueWallet24hChangePercent', 'N/A'),
            'realMc': token_data.get('realMc', 'N/A'),
            'holder': token_data.get('holder', 'N/A'),
            'numberMarkets': token_data.get('numberMarkets', 'N/A'),
            'coingeckoId': token_data.get('extensions', {}).get('coingeckoId', 'N/A'),
            'telegram': token_data.get('extensions', {}).get('telegram', 'N/A'),
            'twitter': token_data.get('extensions', {}).get('twitter', 'N/A'),
            'website': token_data.get('extensions', {}).get('website', 'N/A')
        }

        # Create a DataFrame from the selected data
        df = pd.DataFrame.from_dict(selected_data, orient='index', columns=['Value'])
        df.index.name = 'Attribute'
        df.reset_index(inplace=True)

        # Style the DataFrame
        styled_df = df.style.set_properties(**{
            'white-space': 'pre-wrap',
            'font-family': 'monospace',
            'font-size': '10pt',
            'text-align': 'left'
        }).set_table_styles([
            {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'left')]},
            {'selector': 'td', 'props': [('text-align', 'left')]}
        ])

        # Display the styled DataFrame
        display(HTML("<h3>Selected Token Overview Data</h3>"))
        display(HTML(styled_df.to_html()))
    else:
        display(HTML("<h3>Unexpected response structure. 'data' key not found.</h3>"))
        display(HTML("<pre>" + json.dumps(data, indent=2) + "</pre>"))
else:
    display(HTML(f"<h3>Request failed with status code: {response.status_code}</h3>"))
    display(HTML("<pre>" + response.text + "</pre>"))

,Attribute,Value
0,address,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm
1,decimals,6
2,symbol,$WIF
3,name,dogwifhat
4,price,2.693547
5,liquidity,35275296.015713
6,uniqueWallet30m,590
7,uniqueWallet30mChangePercent,-21.646746
8,uniqueWallet1h,1089
9,uniqueWallet1hChangePercent,-56.229904


Token Trade Data (Single)

In [6]:
address = "So11111111111111111111111111111111111111112"

url = f"https://public-api.birdeye.so/defi/v3/token/trade-data/single?address={address}"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = json.loads(response.text)
    if 'data' in data:
        token_data = data['data']
        
        # Convert the token data to a DataFrame
        df = pd.DataFrame([token_data])
        
        # Transpose the DataFrame for better readability
        df_transposed = df.T.reset_index()
        df_transposed.columns = ['Attribute', 'Value']
        
        # Style the DataFrame
        styled_df = df_transposed.style.set_properties(**{
            'white-space': 'pre-wrap',
            'font-family': 'monospace',
            'font-size': '10pt',
            'text-align': 'left'
        }).set_table_styles([
            {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'left')]},
            {'selector': 'td', 'props': [('text-align', 'left')]}
        ])
        
        # Display the styled DataFrame
        display(HTML("<h3>Token Trade Data</h3>"))
        display(HTML(styled_df.to_html()))
    else:
        print("Unexpected response structure. 'data' key not found.")
        print(json.dumps(data, indent=2))
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

,Attribute,Value
0,address,So11111111111111111111111111111111111111112
1,holder,706664
2,market,49506
3,last_trade_unix_time,1729222745
4,last_trade_human_time,2024-10-18T03:39:05
5,price,152.694997
6,history_30m_price,152.901718
7,price_change_30m_percent,-0.135199
8,history_1h_price,152.843547
9,price_change_1h_percent,-0.097191


Trending List

In [7]:
number_of_tokens = 100

def get_trending_tokens(total_tokens):
    sort_by = "volume24hUSD" # rank, volume24hUSD, liquidity
    sort_type = "desc" # asc, desc
    limit = 20  # API limit per request
    all_tokens = []

    for offset in range(0, total_tokens, limit):
        url = f"https://public-api.birdeye.so/defi/token_trending?sort_by={sort_by}&sort_type={sort_type}&offset={offset}&limit={limit}"

        headers = {
            "accept": "application/json",
            "x-chain": chain,
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'tokens' in data['data']:
                all_tokens.extend(data['data']['tokens'])
            else:
                print(f"Unexpected response structure at offset {offset}")
                print("Response:", response.text)  # Print the response for debugging
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            print("Response:", response.text)  # Print the response for debugging
            break

        if len(all_tokens) >= total_tokens:
            break

    return all_tokens[:total_tokens]

# Get top 100 trending tokens
trending_tokens = get_trending_tokens(number_of_tokens)

if trending_tokens:
    df = pd.DataFrame(trending_tokens)

    # Select and rename columns
    columns_to_display = {
        'symbol': 'Symbol',
        'name': 'Name',
        'price': 'Price',
        'volume24hUSD': 'Volume 24h',
        'liquidity': 'Liquidity',
        'rank': 'Rank'
    }
    df = df[columns_to_display.keys()].rename(columns=columns_to_display)

    # Ensure we have exactly 100 rows
    df = df.head(100)

    # Style the DataFrame
    styled_df = df.style.set_properties(**{
        'white-space': 'nowrap',
        'font-family': 'monospace',
        'font-size': '10pt',
        'text-align': 'right'
    }).set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ]).format({
        'Price': '{:.8f}',
        'Volume 24h': '${:,.2f}',
        'Liquidity': '${:,.2f}',
        'Rank': '{:.0f}'
    })

    # Display the styled DataFrame
    display(HTML(styled_df.to_html()))
else:
    display(HTML("<h3>No trending tokens data available</h3>"))

,Symbol,Name,Price,Volume 24h,Liquidity,Rank
0,AutoLaye,AutoLaye,1.05229146,"$309,660,011.36","$996,178,978.44",601
1,KIMA,Kima,0.83265898,"$208,292,558.13","$216,829,524.29",835
2,GOAT,Goatseus Maximus,0.34836938,"$145,359,358.25","$13,767,075.08",743
3,USDC,USD C,0.99990693,"$143,206,487.18","$46,845,181.74",940
4,USDC,USDC,0.99956712,"$126,930,216.61","$202,188,223.63",712
5,GRA$$,GRA$$,25.77518858,"$109,703,297.18","$157,437,178.77",659
6,AGRY,AGRY,3.07564371,"$99,368,819.49","$131,393,963.99",484
7,RBAX,Rebe1 Bots Ep1c War,3.19371961,"$90,972,364.18","$110,922,777.39",887
8,Babyl0n,Babyl0n,9.34512721,"$82,092,329.86","$62,728,497.20",591
9,$WIF,dogwifhat,2.69354673,"$62,977,585.49","$232,632,018.13",13


New Listings 0-3 days max

In [8]:
days_back = 1  # Change this value to look back fewer days (1 for 1 day, 2 for 2 days, 3 for 3 days max)
hours_back = 0  # Change this value to look back fewer hours within the selected day
                # Note: when hours_back is 1, it looks back for all new tokens within 1 hour.
                # when hours_back is 0, it looks back for all new tokens within the selected minutes.
minutes_back = 1  # Change this value to look back fewer minutes within the selected hour
liquidity_filter = 10000
def iso8601_to_timestamp(iso_string):
    return int(datetime.fromisoformat(iso_string.replace('Z', '+00:00')).timestamp())

def get_new_listings(days_back, hours_back, minutes_back):
    end_time = datetime.now(pytz.UTC)
    
    if days_back == 1:
        start_time = end_time - timedelta(hours=hours_back, minutes=minutes_back)
    elif days_back == 2:
        start_time = end_time - timedelta(days=1, hours=hours_back, minutes=minutes_back)
    elif days_back == 3:
        start_time = end_time - timedelta(days=2, hours=hours_back, minutes=minutes_back)
    else:
        raise ValueError("days_back must be 1, 2, or 3")
    
    all_tokens = []
    current_time = int(end_time.timestamp())
    
    while current_time > int(start_time.timestamp()):
        url = f"https://public-api.birdeye.so/defi/v2/tokens/new_listing?time_to={current_time}&limit=10"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'items' in data['data']:
                new_tokens = data['data']['items']
                if not new_tokens:
                    break
                all_tokens.extend(new_tokens)
                current_time = min(iso8601_to_timestamp(token['liquidityAddedAt']) for token in new_tokens)
            else:
                print(f"Unexpected response structure")
                print("Response:", response.text)
                break
        else:
            print(f"Request failed with status code: {response.status_code}")
            print("Response:", response.text)
            break

        # Respect rate limits
        time.sleep(1)

    # Remove duplicates based on 'address'
    unique_tokens = list({token['address']: token for token in all_tokens}.values())
    
    return unique_tokens

# Get new listings for the past X days, Y hours, and Z minutes
new_listings = get_new_listings(days_back, hours_back, minutes_back)

if new_listings:
    # Display full list of tokens
    df_full = pd.DataFrame(new_listings)
    
    # Convert liquidityAddedAt to datetime
    df_full['liquidityAddedAt'] = pd.to_datetime(df_full['liquidityAddedAt'])
    
    # Sort by liquidityAddedAt in descending order
    df_full = df_full.sort_values('liquidityAddedAt', ascending=False)
    
    # Style the full DataFrame
    styled_df_full = df_full.style.set_properties(**{
        'white-space': 'nowrap',
        'font-family': 'monospace',
        'font-size': '10pt',
        'text-align': 'right'
    }).set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ]).format({
        'liquidity': '{:.8f}',
        'liquidityAddedAt': lambda x: x.strftime('%Y-%m-%d %H:%M:%S')
    })

    # Display the full list
    time_period = f"{days_back} day{'s' if days_back > 1 else ''}, {hours_back} hour{'s' if hours_back > 1 else ''}, and {minutes_back} minute{'s' if minutes_back > 1 else ''}"
    #display(HTML(f"<h3>Full List of New Listings (Past {time_period}) - Total: {len(new_listings)}</h3>"))
    #display(HTML(styled_df_full.to_html()))

    # Apply filters for the second table
    df_filtered = df_full[
        (df_full['name'].notna() & (df_full['name'] != 'None')) & 
        (df_full['logoURI'].notna() & (df_full['logoURI'] != 'None')) & 
        (df_full['liquidity'].astype(float) >= liquidity_filter)
    ]

    # Check if there are any rows left after filtering
    if not df_filtered.empty:
        # Style the filtered DataFrame
        styled_df_filtered = df_filtered.style.set_properties(**{
            'white-space': 'nowrap',
            'font-family': 'monospace',
            'font-size': '10pt',
            'text-align': 'right'
        }).set_table_styles([
            {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]).format({
            'liquidity': '{:.8f}',
            'liquidityAddedAt': lambda x: x.strftime('%Y-%m-%d %H:%M:%S')
        })

        # Display the filtered table
        display(HTML(f"<h3>Filtered New Listings (Past {time_period}) - Total: {len(df_filtered)}</h3>"))
        display(HTML(styled_df_filtered.to_html()))
    else:
        display(HTML("<h3>No listings match the specified criteria</h3>"))
else:
    display(HTML("<h3>No new listings data available</h3>"))

,address,symbol,name,decimals,liquidityAddedAt,logoURI,liquidity
0,2t9rpZEeRkLJE6QvmKjdosoxVPk67TLr9PASasP513gJ,🥚,Egg,9,2024-10-18 03:41:05,https://cdn.dexscreener.com/cms/images/eRkISDi3-kN0iUqL,24753.93094419
2,CXLg1EPX4TxDN42YCqG3qBz4ydFq4gq7SiHFRnSjpump,1SD,1 sol and a dream.,6,2024-10-18 03:38:46,https://ipfs.io/ipfs/QmZhLafKvG31LgjfAHARLSDtL1SH5UoSY4HwoMQS2UAmFA,25043.07323948
5,5gjuXpSMLdJWCJBrvYJEynDzK7ZExadmHHsooEPMpump,MELO,CARAMELO DOG,6,2024-10-18 03:35:55,https://ipfs.io/ipfs/QmPRUg6FKhx7udswLN6aXVoZkMhYPigWGvysRjYt175mQ2,100455.47330395
9,J8eHdjLyc9qA1AZSyeTCCfBv78GznFqkYCzvVC26Asgc,PEPEDENG,Pepe Deng,6,2024-10-18 03:34:32,https://ipfs.io/ipfs/Qmbn9TfMvcqSCYyFEJHhNj5jfzAV5X96vLd43mzUZKWKod,91308.34384628


Top Traders

In [9]:
# Example usage
address = "EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm"
time_frame = "30m"  # 30m, 1h, 4h, 6h, 8h, 12h, 24h
sort_type = "desc"  # asc, desc
sort_by = "volume"  # trade, volume 
            # Note: trade is the number of transactions made, volume is the total volume of transactions made
total_traders = 20  # Change this to the number of top traders you want to retrieve

def get_top_traders(address, time_frame, sort_type, sort_by, total_traders):
    limit = 10
    all_traders = []

    for offset in range(0, total_traders, limit):
        url = f"https://public-api.birdeye.so/defi/v2/tokens/top_traders?address={address}&time_frame={time_frame}&sort_type={sort_type}&sort_by={sort_by}&offset={offset}&limit={limit}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'items' in data['data']:
                all_traders.extend(data['data']['items'])
            else:
                print(f"Unexpected response structure at offset {offset}")
                break
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            break

        if len(all_traders) >= total_traders:
            break

        # Respect rate limits
        time.sleep(1)

    return all_traders[:total_traders]



top_traders = get_top_traders(address, time_frame, sort_type, sort_by, total_traders)

if top_traders:
    df = pd.DataFrame(top_traders)

    # Style the DataFrame
    styled_df = df.style.set_properties(**{
        'white-space': 'nowrap',
        'font-family': 'monospace',
        'font-size': '10pt',
        'text-align': 'right'
    }).set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ]).format({
        'trade': '{:,.0f}',
        'volume': '${:,.2f}',
        'buyVolume': '${:,.2f}',
        'sellVolume': '${:,.2f}'
    })

    # Display the styled DataFrame
    display(HTML(f"<h3>Top {total_traders} Traders for {address} (Time Frame: {time_frame})</h3>"))
    display(HTML(styled_df.to_html()))
else:
    display(HTML("<h3>No top traders data available</h3>"))

,tokenAddress,owner,tags,type,volume,trade,tradeBuy,tradeSell,volumeBuy,volumeSell
0,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,MfDuWeqSHEqTFVYZ7LoexgAK9dxk7cy4DFJWjWMGVWa,[],30m,"$69,343.89",232,76,156,21375.068120,47968.819396
1,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,9AHhmgoxnFq9UP4LDGPRv9z3oxiL3G2E4Dys1LyYNa1c,[],30m,"$46,448.11",96,70,26,33301.229083,13146.878374
2,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,Fd48hVrg4QQznwi8eCJjCfNsbnSxb1ty5rJ5xvSMCXXr,[],30m,"$21,282.86",71,43,28,11645.662861,9637.197435
3,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,4BMibXpmXR9uWRXdB7ZNRhZ5en7h3NkKLg3HBrTTH7TC,[],30m,"$13,604.12",34,2,32,404.247888,13199.870644
4,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,HYe4vSaEGqQKnDrxWDrk3o5H2gznv7qtij5G6NNG8WHd,[],30m,"$13,224.60",87,41,46,5139.186904,8085.417983
5,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,6yKvFA2rBU8CMQG3UNCBQkqxTTFLRE5AMHbxfNWbU7sm,[],30m,"$12,000.42",30,0,30,0.000000,12000.420000
6,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,BYKpCP63FQ9UA64znnrk6QcMExgVXKtpZjeYkEPKwkGR,[],30m,"$9,585.00",2,0,2,0.000000,9585.000000
7,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,JD38n7ynKYcgPpF7k1BhXEeREu1KqptU93fVGy3S624k,[],30m,"$8,480.38",68,43,25,2089.335630,6391.045390
8,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,2NyqyWeJhFcxrQ5EpyrynH4d71tvX4JzAKVtQjVpANUq,[],30m,"$8,000.00",5,0,5,0.000000,8000.000000
9,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,FmZUTtGYfEas53Lj73kbAvAP8ZwG7pcceUtd1FqrATaG,[],30m,"$6,974.48",2,2,0,6974.476030,0.000000


Top Markets for a Token

In [10]:
# Example usage
address = "EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm"
time_frame = "24h"  # 30m, 1h, 4h, 6h, 8h, 12h, 24h 
sort_type = "desc"  # asc, desc
sort_by = "volume24h"  # volume24h, liquidity
total_markets = 50  # Change this to the number of markets you want to retrieve

def get_markets(address, time_frame, sort_type, sort_by, total_markets):
    limit = 10
    all_markets = []

    for offset in range(0, total_markets, limit):
        url = f"https://public-api.birdeye.so/defi/v2/markets?address={address}&time_frame={time_frame}&sort_type={sort_type}&sort_by={sort_by}&offset={offset}&limit={limit}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'items' in data['data']:
                all_markets.extend(data['data']['items'])
            else:
                print(f"Unexpected response structure at offset {offset}")
                break
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            break

        if len(all_markets) >= total_markets:
            break

        # Respect rate limits
        time.sleep(1)

    return all_markets[:total_markets]



markets = get_markets(address, time_frame, sort_type, sort_by, total_markets)

if markets:
    df = pd.DataFrame(markets)

    # Flatten nested dictionaries
    df['base_address'] = df['base'].apply(lambda x: x.get('address'))
    df['base_symbol'] = df['base'].apply(lambda x: x.get('symbol'))
    df['quote_address'] = df['quote'].apply(lambda x: x.get('address'))
    df['quote_symbol'] = df['quote'].apply(lambda x: x.get('symbol'))

    # Select and rename columns based on available data
    columns_to_display = {
        'address': 'Market Address',
        'name': 'Name',
        'base_symbol': 'Base Symbol',
        'quote_symbol': 'Quote Symbol',
        'liquidity': 'Liquidity',
        'volume24h': 'Volume 24h',
        'price': 'Price',
        'trade24h': 'Trades 24h',
        'uniqueWallet24h': 'Unique Wallets 24h',
        'source': 'Source'
    }
    
    # Only include columns that exist in the DataFrame
    available_columns = [col for col in columns_to_display.keys() if col in df.columns]
    df = df[available_columns].rename(columns={col: columns_to_display[col] for col in available_columns})

    # Style the DataFrame
    styled_df = df.style.set_properties(**{
        'white-space': 'nowrap',
        'font-family': 'monospace',
        'font-size': '10pt',
        'text-align': 'right'
    }).set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ]).format({
        'Liquidity': '${:,.2f}',
        'Volume 24h': '${:,.2f}',
        'Price': '{:,.8f}',
        'Trades 24h': '{:,}',
        'Unique Wallets 24h': '{:,}'
    })

    # Display the styled DataFrame
    display(HTML(f"<h3>Top {len(df)} Markets for {address} (Time Frame: {time_frame})</h3>"))
    display(HTML(styled_df.to_html()))
else:
    display(HTML("<h3>No markets data available</h3>"))

,Market Address,Name,Base Symbol,Quote Symbol,Liquidity,Volume 24h,Price,Trades 24h,Unique Wallets 24h,Source
0,D6NdKrKNQPmRZCCnG1GqXtF7MMoHB7qR6GU5TkG59Qz1,SOL-$WIF,SOL,$WIF,"$3,744,345.96","$27,156,073.23",59.26874166,"29,770","4,799",Orca
1,9xERPkyJuPBnffKX2SswG6r25sJMSyD4hTDqgm8d5QoV,$WIF-USDC,$WIF,USDC,"$345,455.12","$6,183,482.36",2.54464386,"22,583","5,229",Lifinity
2,CwGmEwYFo7u5D7vghGwtcCbRToWosytaZa3Ys3JAto6J,$WIF-USDC,$WIF,USDC,"$377,444.08","$5,991,789.11",2.53300000,"2,543",250,Openbook V2
3,BKLhZ5NrFhCjViC4wyAMXBNsJFHbFfYujo3TtUmBxTH3,$WIF-SOL,$WIF,SOL,"$1,668,840.06","$5,097,193.64",0.01693046,"2,888",711,Phoenix
4,G4AGsrzHQWrPRELJdyr8CrWbA3PjvtGDVqMQhZcoaarr,$WIF-SOL,$WIF,SOL,"$342,322.65","$5,083,813.32",0.01686900,"3,056",318,Openbook V2
5,6ojSigXF7nDPyhFRgmn3V9ywhYseKF9J32ZrranMGVSX,$WIF-USDC,$WIF,USDC,"$2,348,872.92","$4,247,445.19",2.55157539,"1,634",595,Phoenix
6,EP2ib6dYdEeqD8MfE2ezHCxX3kP3K2eLKkirfPm5eyMx,$WIF-SOL,$WIF,SOL,"$19,462,576.71","$2,469,470.79",0.01697776,"2,526",526,Raydium
7,5KA7eEtZKSyGHyFwrNQZfFASDXtyPCtYCu5jwMBsQcSQ,$WIF-SOL,$WIF,SOL,"$59,411.36","$1,503,787.75",0.01688197,"6,755","1,605",Lifinity
8,CN8M75cH57DuZNzW5wSUpTXtMrSfXBFScJoQxVCgAXes,$WIF-USDC,$WIF,USDC,"$684,585.17","$915,893.22",2.54162156,"3,734",380,Orca
9,AishPhdJXUHpo3zg2gt5AVfdUg3MPmf8DV8FButvbRJ6,POPCAT-$WIF,POPCAT,$WIF,"$189,275.68","$878,626.28",0.53086904,"3,853",751,Orca


### Wallet APIs

Supported Chains

In [11]:

url = "https://public-api.birdeye.so/v1/wallet/list_supported_chain"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

print(response.text)

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}


Wallet Portfolio Multichain

In [12]:
wallet = d.sol_wallet
multichain = "sol,ethereum,bsc"

url = f"https://public-api.birdeye.so/v1/wallet/multichain_token_list?wallet={wallet}"

headers = {
    "accept": "application/json",
    "x-chains": multichain,
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = json.loads(response.text)
    if 'data' in data:
        all_tokens = []
        for chain, chain_data in data['data'].items():
            if isinstance(chain_data, list):
                for token in chain_data:
                    if isinstance(token, dict):
                        token['chain'] = chain
                        all_tokens.append(token)
            elif isinstance(chain_data, dict):
                chain_data['chain'] = chain
                all_tokens.append(chain_data)
        
        if all_tokens:
            df = pd.DataFrame(all_tokens)
            
            # Reorder columns to have 'chain' first
            columns = ['chain'] + [col for col in df.columns if col != 'chain']
            df = df[columns]
            
            # Style the DataFrame
            styled_df = df.style.set_properties(**{
                'white-space': 'nowrap',
                'font-family': 'monospace',
                'font-size': '10pt',
                'text-align': 'right'
            }).set_table_styles([
                {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
                {'selector': 'td', 'props': [('padding', '5px')]}
            ]).format({
                'price': '${:.8f}',
                'value': '${:.2f}',
                'amount': '{:.8f}'
            })
            
            # Display the styled DataFrame
            display(HTML("<h3>Multichain Wallet Token List</h3>"))
            display(HTML(styled_df.to_html()))
        else:
            print("No tokens found in the wallet.")
    else:
        print("Unexpected response structure. 'data' key not found.")
        print(json.dumps(data, indent=2))
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

,chain,address,decimals,balance,uiAmount,chainId,name,symbol,logoURI,priceUsd,valueUsd,icon
0,items,So11111111111111111111111111111111111111111,9,34723266,0.034723,solana,SOL,SOL,https://wsrv.nl/?w=128&h=128&url=https%3A%2F%2Fraw.githubusercontent.com%2Fsolana-labs%2Ftoken-list%2Fmain%2Fassets%2Fmainnet%2FSo11111111111111111111111111111111111111112%2Flogo.png,152.776104,5.304885,nan
1,items,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm,6,1260929,1.260929,solana,dogwifhat,$WIF,https://wsrv.nl/?w=128&h=128&url=https%3A%2F%2Fbafkreibk3covs5ltyqxa272uodhculbr6kea6betidfwy3ajsav2vjzyum.ipfs.nftstorage.link,2.692078,3.394520,https://wsrv.nl/?w=128&h=128&url=https%3A%2F%2Fbafkreibk3covs5ltyqxa272uodhculbr6kea6betidfwy3ajsav2vjzyum.ipfs.nftstorage.link


Wallet Transaction History (Multichain)

In [13]:
url = f"https://public-api.birdeye.so/v1/wallet/multichain_tx_list?wallet={wallet}"

headers = {
    "accept": "application/json",
    "x-chains": multichain,
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = json.loads(response.text)
    if 'data' in data:
        all_transactions = []
        for chain, chain_data in data['data'].items():
            for tx in chain_data:
                tx['chain'] = chain
                all_transactions.append(tx)
        
        if all_transactions:
            df = pd.DataFrame(all_transactions)
            
            # Convert blockTime to datetime
            df['datetime'] = pd.to_datetime(df['blockTime'])
            
            # Reorder columns to have 'chain' and 'datetime' first
            columns = ['chain', 'datetime'] + [col for col in df.columns if col not in ['chain', 'datetime', 'blockTime']]
            df = df[columns]
            
            # Style the DataFrame
            styled_df = df.style.set_properties(**{
                'white-space': 'nowrap',
                'font-family': 'monospace',
                'font-size': '10pt',
                'text-align': 'right'
            }).set_table_styles([
                {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
                {'selector': 'td', 'props': [('padding', '5px')]}
            ]).format({
                'datetime': lambda x: x.strftime('%Y-%m-%d %H:%M:%S'),
                'fee': '{:,.0f}',
                'blockNumber': '{:,}'
            })
            
            # Display the styled DataFrame
            display(HTML(f"<h3>Multichain Wallet Transaction List (Total: {len(df)} transactions)</h3>"))
            display(HTML(styled_df.to_html()))
        else:
            print("No transactions found for the wallet.")
    else:
        print("Unexpected response structure. 'data' key not found.")
        print(json.dumps(data, indent=2))
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

No transactions found for the wallet.
